In [1]:
# OMDB_insert_id = ObjectId('57b30152c54d2d162e1439c9')


import omdb

import warnings
warnings.filterwarnings('ignore')

import json
import numpy as np

from joblib import Parallel, delayed  
import multiprocessing
num_cores = multiprocessing.cpu_count()

import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.dc_marvel
db.collection_names(include_system_collections=False)


[u'omdb_critics', u'box_office_mojo']

In [2]:
mongo = db.omdb_critics

In [3]:
def getOMDBdata(title):
    if title == 'Fantastic Four(2005)':
        data = omdb.get(title='Fantastic Four', year=2005, fullplot=True, tomatoes=True)
    
    elif title == 'Daredevil':
        data = omdb.get(title='Daredevil', year=2003, fullplot=True, tomatoes=True)
        
    elif title == 'Fantastic Four(2015)':
        data = omdb.get(title='Fantastic Four', year=2015, fullplot=True, tomatoes=True)
    
    elif title == "Marvel's The Avengers":
        data = omdb.get(title='The Avengers', fullplot=True, tomatoes=True)
    
    elif title == 'Supergirl':
        data = omdb.get(title='Supergirl', year=1984, fullplot=True, tomatoes=True)
        
    elif title == "X2: X-Men United":
        data = omdb.get(title='X-men', year=2003, fullplot=True, tomatoes=True)
    
    elif title == "Teenage Mutant Ninja Turtles (2014)":
        data = omdb.get(title='Teenage Mutant Ninja Turtles', year=2014, fullplot=True, tomatoes=True)
    
    elif title == "Teenage Mutant Ninja Turtles (1990)":
        data = omdb.get(title='Teenage Mutant Ninja Turtles', year=1990, fullplot=True, tomatoes=True)
    
    elif title == "Teenage Mutant Ninja Turtles: The Secret of the Ooze":
        data = omdb.get(title="Teenage Mutant Ninja Turtles II: The Secret of the Ooze", fullplot=True, tomatoes=True)
        
        
    elif title == "Men in Black III":
        data = omdb.get(title='Men in Black 3', year=2012, fullplot=True, tomatoes=True)
    
    elif title == "Hercules":
        data = omdb.get(title='Hercules', year=2014, fullplot=True, tomatoes=True) 
        
    elif title == "RIPD":
        data = omdb.get(title='R.I.P.D.', fullplot=True, tomatoes=True) 
    
    
    elif title == "Alien vs Predator":
        data = omdb.get(title="AVP: Alien vs. Predator", fullplot=True, tomatoes=True) 
    
    elif title == "Alien vs Predator Requiem":
        data = omdb.get(title="Aliens vs. Predator: Requiem", fullplot=True, tomatoes=True) 
       
    
    elif title == "Scott Pilgrim vs the World":
        data = omdb.get(title="Scott Pilgrim vs. the World", fullplot=True, tomatoes=True)
        
        
    elif title == "Mighty Morphin' Power Rangers":
        data = omdb.get(title="Mighty Morphin Power Rangers: The Movie", fullplot=True, tomatoes=True)
        
    elif title == "Buffy the Vampire Slayer":
        data = omdb.get(title="Buffy the Vampire Slayer", year= 1992, fullplot=True, tomatoes=True)
    
    
    
    
    else:
        data = omdb.get(title=title, fullplot=True, tomatoes=True)
    
    collect = {}
    collect['awards'] = data['awards']
    collect['imdb_id'] = data['imdb_id']
    collect['imdb_rating'] = data['imdb_rating'] 
    collect['metascore'] = data['metascore']
    collect['plot'] = data['plot']
    collect['poster'] = data['poster']
    collect['release_year'] = data['year']
    collect['release_date'] = data['released']
    collect['RT_consensus'] = data['tomato_consensus']
    collect['RT_perc'] = data['tomato_meter']
    collect['RT_rating'] = data['tomato_rating']
    collect['RT_audience_perc'] = data['tomato_user_meter']
    collect['RT_audience_rating'] = data['tomato_user_rating']
    
    return collect
    

In [4]:
Marvel  = [
    "Howard The Duck",
    "Blade",
    "X-Men",
    "Blade II",
    "Spider-Man",
    "Daredevil",
    "X2: X-Men United",
    "Hulk",
    "The Punisher",
    "Spider-Man 2",
    "Blade: Trinity",
    "Elektra",
    "Fantastic Four(2005)",
    "X-Men: The Last Stand",
    "Ghost Rider","Spider-Man 3",
    "Fantastic Four: Rise of the Silver Surfer","Iron Man",
    "The Incredible Hulk","Punisher: War Zone",
    "X-Men Origins: Wolverine","Iron Man 2",
    "Thor","X-Men: First Class",
    "Captain America: The First Avenger",
    "Ghost Rider: Spirit of Vengeance",
    "Marvel's The Avengers","The Amazing Spider-Man",
    "Iron Man 3","The Wolverine",
    "Thor: The Dark World","Captain America: The Winter Soldier",
    "The Amazing Spider-Man 2","X-Men: Days of Future Past",
    "Guardians of the Galaxy","Big Hero 6",
    "Avengers: Age of Ultron", "Ant-Man",
    "Fantastic Four(2015)", "Deadpool",
    "Captain America: Civil War"
]


Dc = ["Superman","Superman II",
      "Superman III", "Supergirl",
      "Superman IV: The Quest for Peace","Batman",
      "Batman Returns","Batman: Mask of the Phantasm",
      "Batman Forever", "Batman Robin", 
      "Steel", "Catwoman", 
      "Constantine", "Batman Begins",
      "V for Vendetta", "Superman Returns", 
      "The Dark Knight", "Watchmen",
      "The Losers","Jonah Hex","Red",
      "Green Lantern", "The Dark Knight Rises",
      "Man of Steel",
      "Batman v Superman: Dawn of Justice","Red 2"]

Others = ["2 Guns",
"30 Days of Night",
"300",
"300: Rise of an Empire",
"American Splendor",
"Alien vs Predator",
"Alien vs Predator Requiem",
"Black Mask",
"Buffy the Vampire Slayer",
"Bulletproof Monk",
"Bullet to the Head",
"Casper",
"Chronicle",
"Cowboys & Aliens",
"The Crow",
"The Crow:City of Angels",
"Darkman",
"The Diary of a Teenage Girl", "Dick Tracy",
"Dredd",
"From Hell",
"Ghost world",
"The Green Hornet",
"Hancock",
"Hellboy",
"Hellboy II: The Golden Army",
"Hercules",
"A history of violence",
"I, Frankenstein",
"The Incredibles",
"Iron Monkey",
"Josie and the Pussycats",
"Judge Dredd",
"Kick-Ass",
"Kick-Ass 2",
"The League of Extraordinary Gentlemen",
"The Mask",
"The Mask of Zorro",
"Megamind",
"Men in Black",
"Men in Black II",
"Men in Black III",
"Monkeybone",
"Mystery Men",
"Oblivion",
"The Phantom",
"The Powerpuff Girls Movie",
"Mighty Morphin' Power Rangers",
"Priest",
"Richie Rich",    
"RIPD",
"Road to Perdition",
"The Rocketeer",
"Scott Pilgrim vs the World",
"Kingsman: The Secret Service",
"The Shadow",
"Sin City",
"Sin City: A Dame to Kill For",
"Sky High",
"Spawn",
"The Spirit",
"Snowpiercer",
"Super",
"My Super Ex-Girlfriend",
"Superhero Movie",
"Surrogates",
"Tank Girl ",
"Teenage Mutant Ninja Turtles",
"Timecop",
"The Adventures of Tintin",
"TMNT",
"Teenage Mutant Ninja Turtles II: The Secret of the Ooze",
"Teenage Mutant Ninja Turtles (2014)",
"Teenage Mutant Ninja Turtles: Out of the Shadows",
"Teenage Mutant Ninja Turtles III",
"Virus",
"Wanted",
"Whiteout",
"The X-Files: I Want to Believe",
"Zoom",
"The Legend of Zorro"
]

In [5]:
%%capture


DC = Parallel(n_jobs=num_cores)(delayed(getOMDBdata)(title) for title in Dc)
MARVEL = Parallel(n_jobs=num_cores)(delayed(getOMDBdata)(title) for title in Marvel)
OTHERS = Parallel(n_jobs=num_cores)(delayed(getOMDBdata)(title) for title in Others)


M = {}
D = {}
O = {}


for k in range(len(MARVEL)):
    M[Marvel[k]]= MARVEL[k]
    
for k in range(len(DC)):
    D[Dc[k]]= DC[k]
    
for k in range(len(OTHERS)):
    O[Others[k]]= OTHERS[k]

    
Data = {}
Data['Marvel'] = M
Data['DC'] = D
Data['Others'] = O

del M
del D
del O

del Marvel
del MARVEL
del DC
del Dc
del OTHERS
del Others

In [6]:
OMDB_insert_id = mongo.insert_one(Data).inserted_id
db.collection_names(include_system_collections=False)

[u'omdb_critics', u'box_office_mojo']

In [7]:

client.close

<bound method MongoClient.close of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>

In [8]:
OMDB_insert_id 

ObjectId('57b30152c54d2d162e1439c9')